In [7]:
import pandas as pd
import geopandas as gpd
import glob
from pathlib import Path

In [2]:
dissemination_boundary = gpd.read_file("data/DisseminationBoundary/ldb_000a21a_e.shp")

In [3]:
dissemination_boundary.head(2)
dissemination_boundary["DBUID"] = dissemination_boundary["DBUID"].astype(int)

In [4]:
file_paths = glob.glob("data/sam-msa-eng/acs*")
file_paths

['data/sam-msa-eng/acs_cycling.csv',
 'data/sam-msa-eng/acs_public_transit_offpeak.csv',
 'data/sam-msa-eng/acs_walking.csv',
 'data/sam-msa-eng/acs_public_transit_peak.csv']

In [8]:
dfs = []

for file in file_paths:
    df_ = pd.read_csv(file, encoding="latin1")
    df_["type"] = Path(file).stem
    df_["DBUID"] = df_["DBUID"].astype(int)
    dfs.append(df_)

sam = pd.concat(dfs)
type_column = sam.pop("type")
sam.insert(0, "type", type_column)
sam.head(2)

,type,DBUID,CSDUID,CSDNAME,CMAUID,CMANAME,PRUID,PRCODE,acs_idx_hf,acs_idx_emp,acs_idx_srf,acs_idx_psef,acs_idx_ef,acs_idx_caf,acs_lvl_gs-1,acs_lvl_gs-3,acs_lvl_gs-5
0,acs_cycling,10010165001,1001519,St. John's,1.0,St. John's,10,NL,0.022759,0.042675,0.039388,0.0,0.031447,0.024913,5.94245,14.2529,15.599066666666666
1,acs_cycling,10010165002,1001519,St. John's,1.0,St. John's,10,NL,0.021007,0.039877,0.035564,0.0,0.030209,0.022340,7.386466666666666,13.88188333,17.043066666666665


In [9]:
drop = [
    "CSDUID",
    "CMAUID",
    "PRUID_x",
    "PRCODE",
    "DBRPLAMX",
    "DBRPLAMY",
    "LANDAREA",
    "PRUID_y",
    "DGUID",
]

df = sam.merge(dissemination_boundary, left_on="DBUID", right_on="DBUID").drop(
    columns=drop
)

df = df.set_geometry("geometry")
df = df.to_crs(4326)

df = df.sort_values(by=["type", "CSDNAME"])
df = df.round(3)
df.head(2)

,type,DBUID,CSDNAME,CMANAME,acs_idx_hf,acs_idx_emp,acs_idx_srf,acs_idx_psef,acs_idx_ef,acs_idx_caf,acs_lvl_gs-1,acs_lvl_gs-3,acs_lvl_gs-5,geometry
479323,acs_cycling,59331465001,105 Mile Post 2,NaN,0.0,0.0,0.0,0.0,0.0,0.001,..,..,..,"POLYGON ((-121.31321 50.75665, -121.31124 50.7..."
479324,acs_cycling,59331465002,105 Mile Post 2,NaN,0.0,0.0,0.0,0.0,0.0,0.001,..,..,..,"POLYGON ((-121.31321 50.75665, -121.31307 50.7..."


In [11]:
len(df)

1994188

In [12]:
df_flat = df.pivot(
    columns="type",
    index=["DBUID", "CSDNAME", "CMANAME", "geometry"],
    values=[
        "acs_idx_hf",
        "acs_idx_emp",
        "acs_idx_srf",
        "acs_idx_psef",
        "acs_idx_ef",
        "acs_idx_caf",
        "acs_lvl_gs-1",
        "acs_lvl_gs-3",
        "acs_lvl_gs-5",
    ],
)

df_flat.head()

acs_idx_hf  \
type                                                                                acs_cycling   
DBUID       CSDNAME    CMANAME    geometry                                                        
10010165001 St. John's St. John's POLYGON ((-52.7765 47.52924, -52.77704 47.5286...       0.023   
10010165002 St. John's St. John's POLYGON ((-52.77963 47.52964, -52.77899 47.529...       0.021   
10010165006 St. John's St. John's POLYGON ((-52.77976 47.52879, -52.77951 47.528...        0.02   
10010165007 St. John's St. John's POLYGON ((-52.77421 47.52595, -52.77136 47.526...       0.021   
10010166001 St. John's St. John's POLYGON ((-52.77471 47.52662, -52.77366 47.527...       0.022   

                                                                                                                \
type                                                                                acs_public_transit_offpeak   
DBUID       CSDNAME    CMANAME    geometry                                                                       
10010165001 St. John's St. John's POLYGON ((-52.7765 47.52924, -52.77704 47.5286...                      0.012   
10010165002 St. John's St. John's POLYGON ((-52.77963 47.52964, -52.77899 47.529...                      0.014   
10010165006 St. John's St. John's POLYGON ((-52.77976 47.52879, -52.77951 47.528...                      0.013   
10010165007 St. John's St. John's POLYGON ((-52.77421 47.52595, -52.77136 47.526...                      0.014   
10010166001 St. John's St. John's POLYGON ((-52.77471 47.52662, -52.77366 47.527...                      0.015   

                                                                                                             \
type                                                                                acs_public_transit_peak   
DBUID       CSDNAME    CMANAME    geometry                                                                    
10010165001 St. John's St. John's POLYGON ((-52.7765 47.52924, -52.77704 47.5286...                    0.01   
10010165002 St. John's St. John's POLYGON ((-52.77963 47.52964, -52.77899 47.529...                   0.012   
10010165006 St. John's St. John's POLYGON ((-52.77976 47.52879, -52.77951 47.528...                   0.012   
10010165007 St. John's St. John's POLYGON ((-52.77421 47.52595, -52.77136 47.526...                   0.012   
10010166001 St. John's St. John's POLYGON ((-52.77471 47.52662, -52.77366 47.527...                   0.013   

                                                                                                 \
type                                                                                acs_walking   
DBUID       CSDNAME    CMANAME    geometry                                                        
10010165001 St. John's St. John's POLYGON ((-52.7765 47.52924, -52.77704 47.5286...       0.013   
10010165002 St. John's St. John's POLYGON ((-52.77963 47.52964, -52.77899 47.529...       0.011   
10010165006 St. John's St. John's POLYGON ((-52.77976 47.52879, -52.77951 47.528...       0.011   
10010165007 St. John's St. John's POLYGON ((-52.77421 47.52595, -52.77136 47.526...       0.013   
10010166001 St. John's St. John's POLYGON ((-52.77471 47.52662, -52.77366 47.527...       0.013   

                                                                                    acs_idx_emp  \
type                                                                                acs_cycling   
DBUID       CSDNAME    CMANAME    geometry                                                        
10010165001 St. John's St. John's POLYGON ((-52.7765 47.52924, -52.77704 47.5286...       0.043   
10010165002 St. John's St. John's POLYGON ((-52.77963 47.52964, -52.77899 47.529...        0.04   
10010165006 St. John's St. John's POLYGON ((-52.77976 47.52879, -52.77951 47.528...        0.04   
10010165007 St. John's St. John's POLYGON ((-52.77421 47.52595, -52.77136 47.526...       0.039   
1001

In [13]:
df_flat.columns = [
    "_".join(col).strip() if isinstance(col, tuple) else col for col in df_flat.columns
]
df_flat = df_flat.reset_index()
df_flat = df_flat.set_geometry("geometry")
df_flat = df_flat.sort_values(by=["CSDNAME"])
print(len(df_flat))

498547


In [14]:
df_flat.head()

,DBUID,CSDNAME,CMANAME,geometry,acs_idx_hf_acs_cycling,acs_idx_hf_acs_public_transit_offpeak,acs_idx_hf_acs_public_transit_peak,acs_idx_hf_acs_walking,acs_idx_emp_acs_cycling,acs_idx_emp_acs_public_transit_offpeak,...,acs_lvl_gs-1_acs_public_transit_peak,acs_lvl_gs-1_acs_walking,acs_lvl_gs-3_acs_cycling,acs_lvl_gs-3_acs_public_transit_offpeak,acs_lvl_gs-3_acs_public_transit_peak,acs_lvl_gs-3_acs_walking,acs_lvl_gs-5_acs_cycling,acs_lvl_gs-5_acs_public_transit_offpeak,acs_lvl_gs-5_acs_public_transit_peak,acs_lvl_gs-5_acs_walking
479323,59331465001,105 Mile Post 2,NaN,"POLYGON ((-121.31321 50.75665, -121.31124 50.7...",0.0,0.0,0.0,0.0,0.0,0.0,...,..,..,..,..,..,..,..,..,..,..
479324,59331465002,105 Mile Post 2,NaN,"POLYGON ((-121.31321 50.75665, -121.31307 50.7...",0.0,0.0,0.0,0.0,0.0,0.0,...,..,..,..,..,..,..,..,..,..,..
479325,59331465003,105 Mile Post 2,NaN,"POLYGON ((-121.31204 50.73864, -121.31205 50.7...",0.0,0.0,0.0,0.0,0.0,0.0,...,..,..,..,..,..,..,..,..,..,..
479326,59331465004,105 Mile Post 2,NaN,"POLYGON ((-121.31216 50.73497, -121.30969 50.7...",0.0,0.0,0.0,0.0,0.0,0.0,...,..,..,..,..,..,..,..,..,..,..
479327,59331465005,105 Mile Post 2,NaN,"POLYGON ((-121.30699 50.73491, -121.29582 50.7...",0.0,0.0,0.0,0.0,0.0,0.0,...,..,..,..,..,..,..,..,..,..,..


In [44]:
import pyarrow.parquet as pq

# Recommended row group size https://duckdb.org/docs/stable/guides/performance/file_formats.html#microbenchmark-running-aggregation-query-at-different-row-group-sizes
df_flat.to_parquet(
    "spatial_access_measures_wkb.parquet",
    geometry_encoding="WKB",
    schema_version="1.1.0",
    compression="brotli",
    row_group_size=61440,
    sorting_columns=[pq.SortingColumn(1)],
)